In [1]:
import matplotlib.pyplot as plt
import scipy.io as sio
import skimage.io
import numpy as np
import h5py
import sys
import gc

from Data_Extractor import *

/home/admin-u6142160/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
path_train_set = "../Data/090085/Road_Data/motor_trunk_pri_sec_tert_uncl_track/posneg_seg_coord_split_thr1_128_16_train"

# Load training set
train_set = h5py.File(path_train_set, 'r')
train_pos_topleft_coord = np.array(train_set['positive_example'])
train_neg_topleft_coord = np.array(train_set['negative_example'])
train_raw_image = np.array(train_set['raw_image'])
train_road_mask = np.array(train_set['road_mask'])
train_set.close()
gc.collect()


3993

In [2]:
path_cv_set = "../Data/090085/Road_Data/motor_trunk_pri_sec_tert_uncl_track/posneg_seg_coord_split_thr1_128_16_cv"

# Load cross-validation set
CV_set = h5py.File(path_cv_set, 'r')
CV_pos_topleft_coord = np.array(CV_set['positive_example'])
CV_neg_topleft_coord = np.array(CV_set['negative_example'])
CV_raw_image = np.array(CV_set['raw_image'])
CV_road_mask = np.array(CV_set['road_mask'])
CV_set.close()

gc.collect()

43

In [ ]:
X, Y = CV_Data.get_patches(2)
print(X.shape, Y.shape)

In [49]:
class FCN_Data_Extractor (Data_Extractor):

    def __init__(self, raw_image, road_mask, img_size, pos_topleft_coord, neg_topleft_coord, normalization='mean', encoding='one-hot'):

        super(FCN_Data_Extractor, self).__init__(raw_image, road_mask, img_size, pos_topleft_coord, neg_topleft_coord, normalization, encoding)

    def _cal_norm_param(self):
        img_size = self.img_size
        mu = np.zeros((self.band, img_size, img_size))
        pixel_cnt = np.zeros((self.band, img_size, img_size))

        pos_cnt = 0
        neg_cnt = 0

        # Careful! norm params & weight not yet calculated
        for patch, rd_map, _ in self.iterate_data(norm=False, weighted=False):
            patch = patch[0]
            valid_img_mask = np.where(patch != -9999)
            valid_rd_mask = np.where(patch[0] != -9999)

            pixel_cnt[valid_img_mask] = pixel_cnt[valid_img_mask] + 1
            mu[valid_img_mask] = mu[valid_img_mask] + patch[valid_img_mask]

            if self.encoding == 'one-hot':
                pos_cnt += rd_map[0,:,:,1][valid_rd_mask].sum()
                neg_cnt += rd_map[0,:,:,0][valid_rd_mask].sum()
            else:
                pos_cnt += rd_map[valid_rd_mask].sum()
                neg_cnt += -(rd_map[valid_rd_mask]-1).sum()
    
        mu = mu.sum(axis=(1,2)) / pixel_cnt.sum(axis=(1,2))
        self.mu = mu
        self.neg_weight = pos_cnt / (pos_cnt+neg_cnt)
        self.pos_weight = neg_cnt / (pos_cnt+neg_cnt)

        assert (pos_cnt+neg_cnt) == pixel_cnt[0].sum()

        print("mu = ", mu)
        print("class weight [neg= %f, pos= %f]"%(self.neg_weight, self.pos_weight))

        if self.normalization == 'Gaussian':
            std = np.zeros((self.band, img_size, img_size))
            mu_ext = np.repeat(mu, [np.prod(patch[0].shape)]*patch.shape[0]).reshape(patch.shape)

            for patch in self.iterate_raw_image_patches(norm = False):
                patch = patch[0]
                valid_mask = np.where(patch != -9999)
                
                std[valid_mask] = std[valid_mask] + ((patch-mu_ext)**2)[valid_mask]
            
            std = np.sqrt(std.sum(axis=(1,2)) / pixel_cnt.sum(axis=(1,2)))
            self.std = std
            print('std = ', std)


    def _get_raw_patch(self, coord, norm):
        patch  = self.raw_image[:, coord[0]:coord[0]+self.img_size, coord[1]:coord[1]+self.img_size].copy()
        invalid_mask = np.where(patch == -9999)

        if norm:
            if self.normalization == 'mean':
                for channel_num in range(self.band):
                    patch[channel_num] = patch[channel_num] - self.mu[channel_num]
            elif self.normalization == 'Gaussian':
                for channel_num in range(self.band):
                    patch[channel_num] = (patch[channel_num] - self.mu[channel_num]) / self.std[channel_num]
        patch[invalid_mask] = 0
        return patch


    def _get_patch_label(self, coord):
        label = self.road_mask[coord[0]:coord[0]+self.img_size, coord[1]:coord[1]+self.img_size].copy()
        if self.encoding == 'one-hot':
            one_hot_label = np.zeros((self.img_size, self.img_size, 2))
            one_hot_label[:, :, 0][np.where(label == 0)] = 1
            one_hot_label[:, :, 1][np.where(label == 1)] = 1
            return one_hot_label
        return label

    def get_patches(self, batch_size, positive_num = 0, norm = True, wrap_around=True, weighted=True):
        X = []
        Y = []

        if positive_num > 0 and positive_num <= batch_size:
            # pos patches
            X_pos, Y_pos, self.pos_index = self._get_patches_from_topleft_coord (self.pos_topleft_coord,
                                                                     num_of_patches = positive_num,
                                                                     start_index = self.pos_index,
                                                                     norm = norm, 
                                                                     wrap_around=wrap_around)
            
            negative_num = batch_size - positive_num
            X_neg, Y_neg, self.neg_index = self._get_patches_from_topleft_coord (self.neg_topleft_coord,
                                                                     num_of_patches = negative_num,
                                                                     start_index = self.neg_index,
                                                                     norm = norm, 
                                                                     wrap_around=wrap_around)
            
            X.extend (X_pos)
            Y.extend (Y_pos)
            
            X.extend (X_neg)
            Y.extend (Y_neg)
        
        else: # sample batches randomly
            X, Y, self.index = self._get_patches_from_topleft_coord (self.topleft_coordinate,
                                                                      num_of_patches = batch_size,
                                                                      start_index = self.index,
                                                                      norm = norm,
                                                                      wrap_around=wrap_around)

        X = np.array(X)
        Y = np.array(Y)
        W = Y.copy()
    
        if weighted:
            W[:,:,:,0] *= self.neg_weight
            W[:,:,:,1] *= self.pos_weight
            
        return X, Y, W.sum(axis=-1)


    def iterate_data (self, norm=True, weighted=True):
        for coord in self.topleft_coordinate:
            x = np.array([self._get_raw_patch(coord, norm)])
            y = np.array([self._get_patch_label(coord)])
            weight = y.copy()

            if weighted:
                weight[:,:,:,0] *= self.neg_weight
                weight[:,:,:,1] *= self.pos_weight

            yield x, y, weight.sum(axis=-1)


    def iterate_data_with_coord (self, norm=True, weighted=True):
        for coord in self.topleft_coordinate:
            x = np.array([self._get_raw_patch(coord, norm)])
            y = np.array([self._get_patch_label(coord)])
            weight = y.copy()

            if weighted and self.encoding == 'one-hot':
                weight[:,:,:,0] *= self.neg_weight
                weight[:,:,:,1] *= self.pos_weight
            
            yield coord, x, y, weight.sum(axis=-1)

In [6]:
a = np.arange(100).reshape(2,50)
print(a.shape, len(a))

(2, 50) 2


In [50]:
CV_FCN_Data = FCN_Data_Extractor (CV_raw_image, CV_road_mask, 128,
                                  pos_topleft_coord = CV_pos_topleft_coord,
                                  neg_topleft_coord = CV_neg_topleft_coord,
                                  normalization='Gaussian')

(7, 128, 128) (7,) (7, 128, 128) (7,)
mu =  [ 145.49046314  191.39246578  362.50354454  368.10274744 2388.34285012
 1379.97093339  736.84009788]
class weight [neg= 0.025755, pos= 0.974245]
std =  [120.26160916 142.6461666  224.24761472 258.98333524 800.95586468
 840.56659947 560.07518348]


In [14]:
import h5py
import gc

path_train_set = "../Data/090085/Road_Data/motor_trunk_pri_sec_tert_uncl_track/posneg_seg_coord_split_thr1_128_16_train"
path_cv_set = "../Data/090085/Road_Data/motor_trunk_pri_sec_tert_uncl_track/posneg_seg_coord_split_thr1_128_16_cv"

# Load training set
train_set = h5py.File(path_train_set, 'r')
train_pos_topleft_coord = np.array(train_set['positive_example'])
train_neg_topleft_coord = np.array(train_set['negative_example'])
train_raw_image = np.array(train_set['raw_image'])
train_road_mask = np.array(train_set['road_mask'])
train_set.close()

# Load cross-validation set
CV_set = h5py.File(path_cv_set, 'r')
CV_pos_topleft_coord = np.array(CV_set['positive_example'])
CV_neg_topleft_coord = np.array(CV_set['negative_example'])
CV_raw_image = np.array(CV_set['raw_image'])
CV_road_mask = np.array(CV_set['road_mask'])
CV_set.close()

gc.collect()

1407

In [4]:
x, y, w = CV_FCN_Data.get_patches(2)
print(x.shape, y.shape, w.shape)

(2, 7, 128, 128) (2, 128, 128, 2) (2, 128, 128)


In [ ]:
y_f = y.reshape(-1,2)
w_f = w.reshape(-1)
for i in range(y_f.shape[0]):
    print(y_f[i], w_f[i])
    assert sum(np.array([CV_FCN_Data.neg_weight, CV_FCN_Data.pos_weight]) * y_f[i]) == w_f[i] 